In [43]:
import pandas as pd
import os
import preprocessor as p
import string

In [23]:
# !pip install tweet-preprocessor

  Using cached tweet_preprocessor-0.6.0-py3-none-any.whl (27 kB)


In [56]:
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return " "
    elif token.startswith("#"):
        return " "
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return " "
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):

    normTweet = tweet.replace("cannot ", " ").replace("n't ", " ").replace("n 't ", " ").replace("ca n't", " ").replace("ai n't", " ")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace(" i'll ", " ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")
    normTweet = normTweet.replace("RT ", "").replace("rt ", "")
    
    normTweet = normTweet.translate((str.maketrans('','',string.punctuation)))

    tokens = tokenizer.tokenize(normTweet.replace("’", "").replace("…", " "))
    tokens = [word for word in tokens if len(word)>1]
    tokens = [x for x in tokens if not (x.isdigit() 
                                         or x[0] == '-' and x[1:].isdigit())]

    normTweet = " ".join([normalizeToken(token) for token in tokens])
    
    # normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    # normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    # normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [61]:

def generate_texts(mon):
    directory = mon+'/'
    output_dir = 'refined/'+directory
    
    for filename in os.listdir(directory):
        if(filename.endswith('.csv')):
            if(mon == 'Jul' or mon == 'Jun'):
                df = pd.read_csv(directory+filename, encoding='utf-8', usecols = [1, 3], header = None)
                df = df[df[3] == 'en']
                del df[3]
            else:
                df = pd.read_csv(directory+filename, encoding='utf-8', usecols = [1, 4], header = None)
                df = df[df[4] == 'en']
                del df[4]
            df[1] = [normalizeTweet(item) for item in df[1]]
            df.drop_duplicates(subset =[1], keep = False, inplace = True) 
            with open(output_dir+filename[:-4]+'.txt', 'w', encoding='utf-8') as f:
                for item in df[1]:
                    f.write("%s\n" % item)
            # return 0
    return 0

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']

for mon in months:
    df = generate_texts(mon)
    directory = mon+'/'
    
    with open(mon+'_files.txt', 'w', encoding='utf-8') as f:
        for filename in os.listdir(directory):
            if(filename.endswith('.csv')):
                f.write("%s.txt\n" % filename[:-4])


